# CBOW

In [1]:
import pandas as pd 

artigo_treino = pd.read_csv("../Aula 5 /treino.csv")
artigo_teste = pd.read_csv("../Aula 5 /teste.csv")

In [2]:
# pip install -U pip setuptools wheel
# pip install -U spacy
# python -m spacy download pt_core_news_sm

In [3]:
import spacy

nlp = spacy.load("pt_core_news_sm")

In [4]:
texto = "Adoro a cidade de Caldas novas"

doc = nlp(texto)

type(doc)

spacy.tokens.doc.Doc

In [5]:
doc.ents

(Adoro, Caldas novas)

In [6]:
doc[2].is_stop

False

In [7]:
doc[1].is_alpha # é alphanumerico

True

In [8]:
doc[1].is_stop #Verifica se é uma stopWord

True

In [9]:

def tratar_textos(doc): #tirar_stopwords e numeros 
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text)
    if len(tokens_validos) > 2: #word2vec precisa no minimo uma frase com 3 palavras para o contexto
        return " ".join(tokens_validos)

In [10]:
texto = "Adoro 255255 cidade de Caldas Novas!"
doc = nlp(texto)
tratar_textos(doc)

'Adoro cidade Caldas'

In [11]:
textos_para_tratamento = (titulos.lower() for titulos in artigo_treino.title)

textos_tratados = [tratar_textos(doc) for doc in nlp.pipe(textos_para_tratamento,
                                                         batch_size=100,
                                                         n_process= -1)]

In [12]:
titulos_tratados = pd.DataFrame({"titulo": textos_tratados})
titulos_tratados.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron le pen turno frança revés siglas tradic...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço alckmin anuncia queda...
4,queda maio atividade econômica sobe junho bc


In [13]:
from gensim.models import Word2Vec

w2v_modelo = Word2Vec(sg = 0, # 0 para cbow e 1 skip gram 
               window= 2, # quantas palavras ele vai considerar uma antes e uma depois
               vector_size=300, # tamanho do vetor  
               min_count= 5, # eliminar as palavras  não recorrentes
               alpha= 0.3, # taxa de aprendizgem
               min_alpha= 0.007 # minimo de convergencia 
               )

In [14]:
print(len(titulos_tratados))

titulos_tratados = titulos_tratados.dropna().drop_duplicates()
print(len(titulos_tratados))

90000
84466


In [15]:
lista_lista_tokens = [titulo.split(" ") for titulo in titulos_tratados.titulo]

In [16]:
w2v_modelo.build_vocab(lista_lista_tokens)

In [17]:
dir(w2v_modelo)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_check_corpus_sanity',
 '_check_training_sanity',
 '_clear_post_train',
 '_do_train_epoch',
 '_do_train_job',
 '_get_next_alpha',
 '_get_thread_working_mem',
 '_job_producer',
 '_load_specials',
 '_log_epoch_end',
 '_log_epoch_progress',
 '_log_progress',
 '_log_train_end',
 '_raw_word_count',
 '_save_specials',
 '_scan_vocab',
 '_smart_save',
 '_train_epoch',
 '_train_epoch_corpusfile',
 '_worker_loop',
 '_worker_loop_corpusfile',
 'add_lifecycle_event',
 'add_null_word',
 'alpha',
 'batch_words',
 'build_vocab',
 'build_vocab_from_freq',
 'cbow_mean',
 'comment',
 'compute_loss',
 

In [18]:
w2v_modelo.corpus_count

84466

In [19]:
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import Word2Vec

# iniciando a chamada callback
class callback(CallbackAny2Vec):
  def __init__(self):
    self.epoch = 0

  def on_epoch_end(self, model):
    loss = model.get_latest_training_loss()
    if self.epoch == 0:
        print('Loss após a época {}: {}'.format(self.epoch, loss))
    else:
        print('Loss após a época {}: {}'.format(self.epoch, loss- self.loss_previous_step))
    self.epoch += 1
    self.loss_previous_step = loss

In [20]:
w2v_modelo.train(lista_lista_tokens,
                 total_examples= w2v_modelo.corpus_count,
                 epochs= 30,
                 compute_loss= True,
                 callbacks=[callback()])

Loss após a época 0: 462018.09375
Loss após a época 1: 102737.78125
Loss após a época 2: 44945.375
Loss após a época 3: 24812.875
Loss após a época 4: 18739.25
Loss após a época 5: 14626.9375
Loss após a época 6: 12961.9375
Loss após a época 7: 10827.4375
Loss após a época 8: 10010.25
Loss após a época 9: 8979.6875
Loss após a época 10: 8128.0625
Loss após a época 11: 7148.75
Loss após a época 12: 6934.1875
Loss após a época 13: 6352.4375
Loss após a época 14: 6035.125
Loss após a época 15: 6130.375
Loss após a época 16: 5532.625
Loss após a época 17: 5215.0
Loss após a época 18: 5034.875
Loss após a época 19: 4855.9375
Loss após a época 20: 5266.3125
Loss após a época 21: 5051.25
Loss após a época 22: 4508.9375
Loss após a época 23: 4352.5
Loss após a época 24: 4273.25
Loss após a época 25: 4168.125
Loss após a época 26: 4079.5
Loss após a época 27: 3788.6875
Loss após a época 28: 3770.8125
Loss após a época 29: 3150.3125


(14583837, 16207260)

In [21]:
w2v_modelo.wv.most_similar("google")

[('ajudado', 0.42669227719306946),
 ('desenvolve', 0.4007176160812378),
 ('cruzar', 0.38815540075302124),
 ('meme', 0.3784119784832001),
 ('preconceitos', 0.3763431906700134),
 ('françois', 0.37191373109817505),
 ('patentes', 0.3663933575153351),
 ('garcia', 0.36422330141067505),
 ('obstáculo', 0.36026448011398315),
 ('competição', 0.35763129591941833)]

# Skip Gram


In [22]:
from gensim.models import Word2Vec

w2v_modelo_sg = Word2Vec(sg = 1, # 0 para cbow e 1 skip gram 
               window= 5, # quantas palavras ele vai considerar uma antes e uma depois
               vector_size=300, # tamanho do vetor  
               min_count= 5, # eliminar as palavras  não recorrentes
               alpha= 0.3, # taxa de aprendizgem
               min_alpha= 0.007 # minimo de convergencia 
               )

w2v_modelo_sg.build_vocab(lista_lista_tokens)



In [23]:
w2v_modelo_sg.train(lista_lista_tokens,
                 total_examples= w2v_modelo_sg.corpus_count,
                 epochs= 30,
                 compute_loss= True,
                 callbacks=[callback()])

Loss após a época 0: 589820.875
Loss após a época 1: 55995.0625
Loss após a época 2: 27323.125
Loss após a época 3: 16991.75
Loss após a época 4: 13608.5
Loss após a época 5: 10883.125
Loss após a época 6: 8703.375
Loss após a época 7: 6927.6875
Loss após a época 8: 6219.75
Loss após a época 9: 5386.8125
Loss após a época 10: 4905.0625
Loss após a época 11: 3961.8125
Loss após a época 12: 4095.5625
Loss após a época 13: 3471.125
Loss após a época 14: 2694.125
Loss após a época 15: 2303.25
Loss após a época 16: 2053.3125
Loss após a época 17: 2081.3125
Loss após a época 18: 1720.5
Loss após a época 19: 1530.125
Loss após a época 20: 1408.375
Loss após a época 21: 1249.375
Loss após a época 22: 1155.6875
Loss após a época 23: 938.3125
Loss após a época 24: 988.25
Loss após a época 25: 738.25
Loss após a época 26: 767.4375
Loss após a época 27: 644.9375
Loss após a época 28: 647.75
Loss após a época 29: 490.1875


(14584562, 16207260)

In [24]:
w2v_modelo_sg.wv.most_similar("google")

[('esperam', 0.662084698677063),
 ('prejuízo', 0.640738308429718),
 ('corajosa', 0.63727205991745),
 ('piratas', 0.6354288458824158),
 ('aval', 0.6324977278709412),
 ('falas', 0.6290485858917236),
 ('iniciada', 0.6269177198410034),
 ('acabar', 0.6228968501091003),
 ('explode', 0.6205697655677795),
 ('jornalistas', 0.6179621815681458)]

## Exportando modelos

In [25]:
w2v_modelo.wv.save_word2vec_format("./meu_modelo_cbow.txt", binary= False)

In [26]:
w2v_modelo_sg.wv.save_word2vec_format("./meu_modelo_skip_gram.txt", binary= False)

In [27]:
from gensim.models import KeyedVectors

meu_modelo_cbow = KeyedVectors.load_word2vec_format("meu_modelo_cbow.txt")
meu_modelo_sg = KeyedVectors.load_word2vec_format("meu_modelo_skip_gram.txt")


In [28]:
meu_modelo_cbow.most_similar("ana")

[('femininos', 0.4863456189632416),
 ('legítima', 0.4783555865287781),
 ('aguinaldo', 0.4683089554309845),
 ('dizendo', 0.43608853220939636),
 ('ganhará', 0.4015822410583496),
 ('alongar', 0.40126514434814453),
 ('britney', 0.39318206906318665),
 ('bauza', 0.39195120334625244),
 ('farra', 0.3910873830318451),
 ('herdeiro', 0.3878246545791626)]

# Classificador

In [29]:
# def tokenizador(texto):
#   tokens_validos = []
#   doc = nlp(texto)
#   for token in doc:
#     e_valido = not token.is_stop and token.is_alpha
#     if e_valido:
#       tokens_validos.append(token.text.lower())

#   return tokens_validos

#COMENTEI ESSE TOKENIZADOR POR QUE ELE NÃO FUNCIONA BEM COM SPACY QUE USA PROCESSAMENTO EM LOTES

# import numpy as np

# def combinacao_de_vetores_por_soma(palavras_numeros,modelo):
#   vetor_resultante = np.zeros(300)
#   for pn in palavras_numeros:
#     try: 
#       vetor_resultante =+ modelo.get_vector(pn)
#     except KeyError:
#       pass
#   return vetor_resultante
# def matriz_vetores(textos,modelo):
#     x = len(textos)
#     y = 300
#     matriz = np.zeros((x,y))

#     for i in range(x):
#         palavras = tokenizador(textos.iloc[i])
#         matriz[i]= combinacao_de_vetores_por_soma(palavras,modelo)
#     return matriz
# matriz_vetores_treino_cbow = matriz_vetores(artigo_treino.title,meu_modelo)
# matriz_vetores_teste_cbow = matriz_vetores(artigo_teste.title,meu_modelo)

# COMENTEI O RESTO DO CÓDIGO PQ += ESTA ERRADO NA FUNÇÃO  vetor_resultante =+ modelo.get_vector(pn)
#  E A LOGICA NÃO IA FUNCIONAR FAZENDO O TOKENIZADOR DIFERENTE ENTÃO PRECISAVA DE MODIFICAÇÃO
# O USO DO SPACY DEVERIA TER SIDO COMO NA FUNÇÃO NO COMEÇO DO NOTEBOOK TRATAR TEXTO


import spacy

nlp = spacy.load(
    "pt_core_news_sm",
    disable=["parser", "ner", "tagger", "textcat"]
)

def tokenizador(doc):
    return [
        token.text.lower()
        for token in doc
        if token.is_alpha and not token.is_stop
    ]

import numpy as np

def combinacao_de_vetores_por_soma(palavras, modelo):
    vetor_resultante = np.zeros(300)
    for pn in palavras:
        try:
            vetor_resultante += modelo.get_vector(pn)
        except KeyError:
            pass
    return vetor_resultante


def matriz_vetores(docs, modelo):
    x = len(docs)
    y = 300
    matriz = np.zeros((x, y))

    for i, doc in enumerate(docs):
        palavras = tokenizador(doc)
        matriz[i] = combinacao_de_vetores_por_soma(palavras, modelo)

    return matriz




In [30]:
docs_treino = list(nlp.pipe(artigo_treino.title))
docs_teste  = list(nlp.pipe(artigo_teste.title))

matriz_vetores_treino_cbow = matriz_vetores(docs_treino, meu_modelo_cbow)
matriz_vetores_teste_cbow  = matriz_vetores(docs_teste, meu_modelo_cbow)
matriz_vetores_treino_sg = matriz_vetores(docs_treino, meu_modelo_sg)
matriz_vetores_teste_sg  = matriz_vetores(docs_teste, meu_modelo_sg)

# DA FORMA COMENTADA DEMOROU 20 MINUTOS PARA RODAR E AQUI 2MIN


In [31]:
type(matriz_vetores_teste_cbow)

numpy.ndarray

In [32]:
print(matriz_vetores_treino_cbow.shape)
print(matriz_vetores_teste_cbow.shape)

(90000, 300)
(20513, 300)


In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def classificador(x_treino,y_treino,x_teste, y_teste):
    LR = LogisticRegression(max_iter= 800)
    LR.fit(x_treino,y_treino)
    categorias = LR.predict(x_teste)
    CR = classification_report(y_teste,categorias)
    print(CR)



In [34]:
LR_cbow = classificador(matriz_vetores_treino_cbow,
                        artigo_treino.category,
                        matriz_vetores_teste_cbow,
                        artigo_teste.category)

KeyboardInterrupt: 

In [ ]:
LR_skip_gram = classificador(matriz_vetores_treino_sg,
                        artigo_treino.category,
                        matriz_vetores_teste_sg,
                        artigo_teste.category)